In [6]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [8]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
file_path = Path('lending_data.csv')
df_lending_data = pd.read_csv(file_path)

# Review the DataFrame
df_lending_data.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [9]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = df_lending_data['loan_status']
# Separate the X variable, the features
X = df_lending_data.drop(columns='loan_status')

In [10]:
# Review the y variable Series
y.loc[:5]

0    0
1    0
2    0
3    0
4    0
5    0
Name: loan_status, dtype: int64

In [11]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [12]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [13]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=1,
                                                    stratify=y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [14]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)
classifier

# Fit the model using training data
classifier.fit(X_train,y_train)

LogisticRegression(max_iter=200, random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [15]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
results = pd.DataFrame({'Prediction': predictions,'Actual':y_test}).reset_index(drop=True)
results.head(10)

,Prediction,Actual
0,0,0
1,0,1
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [16]:
# Print the balanced_accuracy score of the model
print(f'The balanced_accuracy score is: {balanced_accuracy_score(y_test,predictions)}')

The balanced_accuracy score is: 0.9442676901753825


In [17]:
# Generate a confusion matrix for the model
confusion_matrix(y_test,predictions)

array([[18679,    80],
       [   67,   558]])

In [18]:
# Print the classification report for the model
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      0.89      0.88       625

    accuracy                           0.99     19384
   macro avg       0.94      0.94      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Looking at the classification report, both the precision and recall scores for `0` (healthy loan) are 100%, which means the model can predict this class extramley well. And for the `1` (high-risk loan), even though the scores are less, they are still very closed to 90%, which means the model can still identify those high-risk loan in a very sensitive way. Overall, we have an accuracy score of 99%. So we believe the logistic regression model can predict both labels very well.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points.

In [36]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
random_oversampler = RandomOverSampler(random_state=1)
random_oversampler
# Fit the original training data to the random_oversampler model
X_train_resampled, y_train_resampled = random_oversampler.fit_resample(X_train,y_train)

In [37]:
# Count the distinct values of the resampled labels data
print(y_train_resampled.value_counts())

0    56277
1    56277
Name: loan_status, dtype: int64


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [38]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier_resampled = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)
classifier_resampled

# Fit the model using the resampled training data
classifier_resampled.fit(X_train_resampled,y_train_resampled)

# Make a prediction using the testing data
predictions_resampled = classifier_resampled.predict(X_test)
results_resampled = pd.DataFrame({'Prediction_resampled': predictions_resampled,'Actual_resampled':y_test}).reset_index(drop=True)
results_resampled.head(10)

,Prediction_resampled,Actual_resampled
0,0,0
1,1,1
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [39]:
# Print the balanced_accuracy score of the model
print(f'The balanced_accuracy score is: {balanced_accuracy_score(y_test,predictions_resampled)}')

The balanced_accuracy score is: 0.9959744975744975


In [40]:
# Generate a confusion matrix for the model
confusion_matrix(y_test,predictions_resampled)

array([[18668,    91],
       [    2,   623]])

In [41]:
# Print the classification report for the model
print(classification_report(y_test, predictions_resampled))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      1.00      0.93       625

    accuracy                           1.00     19384
   macro avg       0.94      1.00      0.96     19384
weighted avg       1.00      1.00      1.00     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Fitting with the oversampled data, the scores in classification report is increased, which means the prediction accuracy of the logistic regression model is better. Therefore, the model predicts both the healthy loan and high-risk loan better while fitting with oversampled data.